Notebook to load the result.json from a random search / grid search hyperparameter selection section

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import json
from pathlib import Path
import sys
import os
import re

sys.path.append('../')
from eval_metrics import METRICS

In [2]:
run_name = 'ctm_random_search_20240123_002111'
run_dir = Path(run_name)
run_result_json_path = run_dir.joinpath('result.json')
run_config_json_path = run_dir.joinpath('config.json')

metrics_names = [m.value for m in METRICS]

In [3]:
with open(run_result_json_path) as f:
    run_result = json.load(f)

with open(run_config_json_path) as f:
    run_config = json.load(f)

run_result

{'best_metric': -0.01634058093071984,
 'best_model_checkpoint': 'ctm_random_search_20240123_002111/ctmsb_model_name_or_path_all-mpnet-base-v2_ctm_num_epochs_100_ctm_n_components_10_ctm_hidden_sizes_(200, 200)_cvect_ngram_range_[1, 2]_cvect_max_features_2500',
 'best_hyperparameters': {'sbert_params': {'model_name_or_path': 'all-mpnet-base-v2'},
  'countvect_params': {'ngram_range': [1, 2], 'max_features': 2500},
  'ctm_params': {'dropout': 0.2,
   'lr': 0.002,
   'momentum': 0.99,
   'solver': 'adam',
   'num_epochs': 100,
   'n_components': 10,
   'hidden_sizes': [200, 200],
   'bow_size': 1227,
   'contextual_size': 768}},
 'monitor_type': 'c_npmi',
 'log_history': [{'c_npmi': -0.018172467372008744,
   'c_v': 0.45579005185159394,
   'u_mass': -0.05990959260352226,
   'c_uci': -1.3314675636930327,
   'topic_diversity': 0.675,
   'inverted_rbo': 0.9513249844553008,
   'pairwise_jaccard_similarity': 0.034047684703542594,
   'model_name': 'sb_model_name_or_path_all-roberta-large-v1_ctm_n

In [4]:
# focus on the log history
# create a dataframe from the log history

log_history = run_result['log_history']

# from: https://www.freecodecamp.org/news/how-to-flatten-a-dictionary-in-python-in-4-different-ways/
from collections.abc import MutableMapping

def _flatten_dict_gen(d, parent_key, sep):
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            yield from flatten_dict(v, new_key, sep=sep).items()
        else:
            yield new_key, v


def flatten_dict(d: MutableMapping, parent_key: str = '', sep: str = '.'):
    return dict(_flatten_dict_gen(d, parent_key, sep))

# for each dictionary in the log_history list
# convert them to a flattened dictionary
# then append to a list
log_history_flattened = [flatten_dict(log, sep='.') for log in log_history]
log_history_flattened


[{'c_npmi': -0.018172467372008744,
  'c_v': 0.45579005185159394,
  'u_mass': -0.05990959260352226,
  'c_uci': -1.3314675636930327,
  'topic_diversity': 0.675,
  'inverted_rbo': 0.9513249844553008,
  'pairwise_jaccard_similarity': 0.034047684703542594,
  'model_name': 'sb_model_name_or_path_all-roberta-large-v1_ctm_num_epochs_50_ctm_n_components_20_ctm_hidden_sizes_(200, 200, 200)_cvect_ngram_range_[1, 1]_cvect_max_features_1500',
  'hyperparameters.sbert_params.model_name_or_path': 'all-roberta-large-v1',
  'hyperparameters.countvect_params.ngram_range': [1, 1],
  'hyperparameters.countvect_params.max_features': 1500,
  'hyperparameters.ctm_params.dropout': 0.2,
  'hyperparameters.ctm_params.lr': 0.002,
  'hyperparameters.ctm_params.momentum': 0.99,
  'hyperparameters.ctm_params.solver': 'adam',
  'hyperparameters.ctm_params.num_epochs': 50,
  'hyperparameters.ctm_params.n_components': 20,
  'hyperparameters.ctm_params.hidden_sizes': [200, 200, 200],
  'hyperparameters.ctm_params.bow_s

In [5]:
# create a dataframe from the list of flattened dictionaries
log_history_df = pd.DataFrame(log_history_flattened)
log_history_df

,c_npmi,c_v,u_mass,c_uci,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_name,hyperparameters.sbert_params.model_name_or_path,hyperparameters.countvect_params.ngram_range,hyperparameters.countvect_params.max_features,hyperparameters.ctm_params.dropout,hyperparameters.ctm_params.lr,hyperparameters.ctm_params.momentum,hyperparameters.ctm_params.solver,hyperparameters.ctm_params.num_epochs,hyperparameters.ctm_params.n_components,hyperparameters.ctm_params.hidden_sizes,hyperparameters.ctm_params.bow_size,hyperparameters.ctm_params.contextual_size
0,-0.018172,0.455790,-0.059910,-1.331468,0.675,0.951325,0.034048,sb_model_name_or_path_all-roberta-large-v1_ctm...,all-roberta-large-v1,"[1, 1]",1500,0.2,0.002,0.99,adam,50,20,"[200, 200, 200]",1500,1024
1,-0.031282,0.427796,-0.026496,-1.584586,0.860,0.970501,0.020393,sb_model_name_or_path_all-roberta-large-v1_ctm...,all-roberta-large-v1,"[1, 2]",2000,0.2,0.002,0.99,adam,20,10,"[200, 200]",1227,1024
2,-0.016341,0.443902,-0.019198,-1.180072,0.820,0.965105,0.026472,sb_model_name_or_path_all-mpnet-base-v2_ctm_nu...,all-mpnet-base-v2,"[1, 2]",2500,0.2,0.002,0.99,adam,100,10,"[200, 200]",1227,768


In [11]:
# sort by c_npmi
aaa = log_history_df.sort_values(by='c_npmi', ascending=False)
aaa

,c_npmi,c_v,u_mass,c_uci,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_name,hyperparameters.sbert_params.model_name_or_path,hyperparameters.vocab_tokenizer_params.ngram_range,...,hyperparameters.umap_params.low_memory,hyperparameters.umap_params.random_state,hyperparameters.hdbscan_params.metric,hyperparameters.hdbscan_params.prediction_data,hyperparameters.hdbscan_params.min_samples,hyperparameters.hdbscan_params.min_cluster_size,hyperparameters.bertopic_params.language,hyperparameters.bertopic_params.calculate_probabilities,hyperparameters.bertopic_params.top_n_words,hyperparameters.bertopic_params.nr_topics
11,0.159360,0.736042,-0.215378,1.572884,0.680000,0.971925,0.029161,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,all-mpnet-base-v1,"[1, 2]",...,False,None,euclidean,True,40,180,english,True,10,71
17,0.148927,0.714929,-0.218321,1.494394,0.678750,0.976178,0.023130,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,all-mpnet-base-v1,"[1, 2]",...,False,None,euclidean,True,10,120,english,True,10,81
15,0.143404,0.704631,-0.222557,1.398259,0.688000,0.968158,0.027177,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,all-mpnet-base-v1,"[1, 2]",...,False,None,euclidean,True,30,90,english,True,10,51
4,0.137412,0.713508,-0.226645,1.238540,0.718421,0.961122,0.035787,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,10,90,english,True,10,41
2,0.132052,0.719587,-0.083229,1.092262,0.660000,0.886554,0.113510,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,all-mpnet-base-v1,"[1, 2]",...,False,None,euclidean,True,40,200,english,True,10,11
1,0.111151,0.744181,-0.152728,0.891481,0.605085,0.945372,0.060758,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,40,180,english,True,20,61
10,0.110444,0.719416,-0.227431,0.941981,0.680769,0.977346,0.024298,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,30,120,english,True,20,81
16,0.101151,0.724493,-0.216620,0.738231,0.640000,0.946919,0.055351,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,all-mpnet-base-v1,"[1, 2]",...,False,None,euclidean,True,50,120,english,True,20,31
9,0.100998,0.761304,-0.160130,0.824367,0.674359,0.971981,0.031359,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,50,180,english,True,30,81
23,0.100161,0.719013,-0.165695,0.755407,0.636735,0.953011,0.048740,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,50,120,english,True,20,51


In [12]:
aaa[aaa['hyperparameters.sbert_params.model_name_or_path'] == 'all-MiniLM-L12-v1']

,c_npmi,c_v,u_mass,c_uci,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_name,hyperparameters.sbert_params.model_name_or_path,hyperparameters.vocab_tokenizer_params.ngram_range,...,hyperparameters.umap_params.low_memory,hyperparameters.umap_params.random_state,hyperparameters.hdbscan_params.metric,hyperparameters.hdbscan_params.prediction_data,hyperparameters.hdbscan_params.min_samples,hyperparameters.hdbscan_params.min_cluster_size,hyperparameters.bertopic_params.language,hyperparameters.bertopic_params.calculate_probabilities,hyperparameters.bertopic_params.top_n_words,hyperparameters.bertopic_params.nr_topics
4,0.137412,0.713508,-0.226645,1.238540,0.718421,0.961122,0.035787,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,10,90,english,True,10,41
1,0.111151,0.744181,-0.152728,0.891481,0.605085,0.945372,0.060758,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,40,180,english,True,20,61
10,0.110444,0.719416,-0.227431,0.941981,0.680769,0.977346,0.024298,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,30,120,english,True,20,81
9,0.100998,0.761304,-0.160130,0.824367,0.674359,0.971981,0.031359,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,50,180,english,True,30,81
23,0.100161,0.719013,-0.165695,0.755407,0.636735,0.953011,0.048740,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,50,120,english,True,20,51
7,0.099775,0.740856,-0.177363,0.686020,0.575862,0.913845,0.091492,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,20,200,english,True,20,31
22,0.092475,0.629878,-0.465147,0.468985,0.840000,0.984671,0.013068,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,10,30,english,True,10,31
0,0.092217,0.753588,-0.150548,0.645419,0.606250,0.951025,0.053354,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,30,180,english,True,30,51
5,0.091976,0.753699,-0.143123,0.690079,0.657303,0.976477,0.025502,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,50,150,english,True,30,91
20,0.091074,0.745403,-0.152650,0.678093,0.668116,0.968834,0.033474,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,all-MiniLM-L12-v1,"[1, 2]",...,False,None,euclidean,True,50,150,english,True,30,71


In [20]:
# only show the columns within the search_space
search_space = run_config['search_space']

# flatten the dictionary
search_space_flattened = flatten_dict({'hyperparameters': search_space}, sep='.')
search_space_flattened

{'hyperparameters.sbert_params.model_name_or_path': ['all-MiniLM-L12-v1',
  'all-mpnet-base-v1'],
 'hyperparameters.hdbscan_params.min_cluster_size': [30,
  60,
  90,
  120,
  150,
  180,
  200],
 'hyperparameters.hdbscan_params.min_samples': [10, 20, 30, 40, 50],
 'hyperparameters.bertopic_params.top_n_words': [10, 20, 30],
 'hyperparameters.bertopic_params.nr_topics': [10,
  20,
  30,
  40,
  50,
  60,
  70,
  80,
  90,
  100]}

In [23]:
bbb = log_history_df[['model_name'] + metrics_names + list(search_space_flattened.keys())]
bbb

,model_name,u_mass,c_v,c_uci,c_npmi,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,hyperparameters.sbert_params.model_name_or_path,hyperparameters.hdbscan_params.min_cluster_size,hyperparameters.hdbscan_params.min_samples,hyperparameters.bertopic_params.top_n_words,hyperparameters.bertopic_params.nr_topics
0,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.150548,0.753588,0.645419,0.092217,0.606250,0.951025,0.053354,all-MiniLM-L12-v1,180,30,30,51
1,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.152728,0.744181,0.891481,0.111151,0.605085,0.945372,0.060758,all-MiniLM-L12-v1,180,40,20,61
2,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,-0.083229,0.719587,1.092262,0.132052,0.660000,0.886554,0.113510,all-mpnet-base-v1,200,40,10,11
3,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.303749,0.648377,0.202981,0.071528,0.768571,0.987691,0.011415,all-MiniLM-L12-v1,30,10,20,71
4,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.226645,0.713508,1.238540,0.137412,0.718421,0.961122,0.035787,all-MiniLM-L12-v1,90,10,10,41
5,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.143123,0.753699,0.690079,0.091976,0.657303,0.976477,0.025502,all-MiniLM-L12-v1,150,50,30,91
6,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.217763,0.700117,0.289676,0.071857,0.750000,0.983080,0.016910,all-MiniLM-L12-v1,30,40,30,51
7,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.177363,0.740856,0.686020,0.099775,0.575862,0.913845,0.091492,all-MiniLM-L12-v1,200,20,20,31
8,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,-0.189626,0.709542,0.435725,0.078202,0.702899,0.978370,0.019305,all-mpnet-base-v1,90,40,30,71
9,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.160130,0.761304,0.824367,0.100998,0.674359,0.971981,0.031359,all-MiniLM-L12-v1,180,50,30,81


In [24]:
bbb.sort_values(by='c_npmi', ascending=False)

,model_name,u_mass,c_v,c_uci,c_npmi,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,hyperparameters.sbert_params.model_name_or_path,hyperparameters.hdbscan_params.min_cluster_size,hyperparameters.hdbscan_params.min_samples,hyperparameters.bertopic_params.top_n_words,hyperparameters.bertopic_params.nr_topics
11,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,-0.215378,0.736042,1.572884,0.159360,0.680000,0.971925,0.029161,all-mpnet-base-v1,180,40,10,71
17,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,-0.218321,0.714929,1.494394,0.148927,0.678750,0.976178,0.023130,all-mpnet-base-v1,120,10,10,81
15,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,-0.222557,0.704631,1.398259,0.143404,0.688000,0.968158,0.027177,all-mpnet-base-v1,90,30,10,51
4,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.226645,0.713508,1.238540,0.137412,0.718421,0.961122,0.035787,all-MiniLM-L12-v1,90,10,10,41
2,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,-0.083229,0.719587,1.092262,0.132052,0.660000,0.886554,0.113510,all-mpnet-base-v1,200,40,10,11
1,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.152728,0.744181,0.891481,0.111151,0.605085,0.945372,0.060758,all-MiniLM-L12-v1,180,40,20,61
10,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.227431,0.719416,0.941981,0.110444,0.680769,0.977346,0.024298,all-MiniLM-L12-v1,120,30,20,81
16,sb_model_name_or_path_all-mpnet-base-v1_hs_min...,-0.216620,0.724493,0.738231,0.101151,0.640000,0.946919,0.055351,all-mpnet-base-v1,120,50,20,31
9,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.160130,0.761304,0.824367,0.100998,0.674359,0.971981,0.031359,all-MiniLM-L12-v1,180,50,30,81
23,sb_model_name_or_path_all-MiniLM-L12-v1_hs_min...,-0.165695,0.719013,0.755407,0.100161,0.636735,0.953011,0.048740,all-MiniLM-L12-v1,120,50,20,51
